# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/joe/Projects/Udacity/DataEngineering/Data_Modeling_With_Apache_Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf-8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# create event_datafile_full.csv
# will be used as an intermediate step for pushing all data found in the event_data directory into the 
# Apache Cassandra keyspace tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf-8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
# create an enum to reference column names of event data csv file 
# to avoid magic numbers when assembling INSERT statements later in this notebook
from enum import Enum

class EventData(Enum):
    artist = 0
    firstName = 1
    gender = 2
    itemInSession = 3
    lastName = 4
    length = 5
    level = 6
    location = 7
    sessionId = 8
    song = 9
    userId = 10

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# Establish connection to local instance of Apache Cassandra
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except exception as e:
    print(e)

#### Create Keyspace

In [8]:
# create the keyspace
query_keyspace_create = """
CREATE KEYSPACE IF NOT EXISTS sparkifydb 
WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}
"""

try:
    session.execute(query_keyspace_create)
except exception as e:
    print(e)

#### Set Keyspace

In [9]:
# set the keyspace
try:
    session.set_keyspace('sparkifydb')
except exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [10]:
# table to handle query 1
# fields: artist, song, length, sessionid, itemInSession
# Partition key = sessionid, itemInSession (composite)
# Clustering key = None
create_query1_table = """
CREATE TABLE IF NOT EXISTS query1 (
    sessionId int,
    itemInSession int,
    song text,
    artist text,
    length decimal,
    PRIMARY KEY (
        sessionId,
        itemInSession
    )
)
"""
try:
    session.execute(create_query1_table)
except Exception as e:
    print(repr(e))

In [11]:
# table to handle query 2
# fields: artist, song, user first name, user last name, userId, sessionId, itemInSession (for sorting)
# Partition key =  userId, sessionId (composite)
# Clustering Key = itemInSession (sort by this)
create_query2_table = """
CREATE TABLE IF NOT EXISTS query2 (
    userId int,
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    firstName text,
    lastName text,
    PRIMARY KEY (
        (
            userId,
            sessionId
        ),
        itemInSession
    )
)
"""
try:
    session.execute(create_query2_table)
except Exception as e:
    print(repr(e))

In [12]:
# table to handle query 3
# fields: user first name, user last name, song
# Partition key =  song, userId (composite, order matters since we're going to use song in our WHERE clause)
# Clustering Key = none
create_query3_table = """
CREATE TABLE IF NOT EXISTS query3 (
    song text,
    userId int,
    firstName text,
    lastName text,
    PRIMARY KEY (
        song,
        userId
    )
)
"""
try:
    session.execute(create_query3_table)
except Exception as e:
    print(repr(e))

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [13]:
# define query1
query1 = """
SELECT artist, song, length 
FROM query1 
WHERE sessionId=338 AND itemInSession=4
"""

In [14]:
# insert data into data to perform query1
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO query1 (
            sessionId,
            itemInSession,
            artist,
            song,
            length
        )
        VALUES (?, ?, ?, ?, ?)
        """
        try:
            s = session.prepare(query)
            bound = s.bind((int(line[EventData.sessionId.value]), 
                            int(line[EventData.itemInSession.value]), 
                            line[EventData.artist.value], 
                            line[EventData.song.value], 
                            "{:.5f}".format(float(line[EventData.length.value]))))
            session.execute(bound)
        except Exception as e:
            print(repr(e))
            print(bound)

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
# execute query1
try:
    rows = session.execute(query1)
    for row in rows:
        print(row)
except Exception as e:
    print(repr(e))

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=Decimal('495.30730'))


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [16]:
# define query2
query2 = """
SELECT artist, song, firstName, lastName 
FROM query2 
WHERE userId=10 AND sessionId=182
"""

In [17]:
# insert data into table to perform query2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = f"""
        INSERT INTO query2 (
            userId, 
            sessionId, 
            itemInSession,
            artist,
            song,
            firstName,
            lastName
        ) 
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """
        try:
            s = session.prepare(query)
            bound = s.bind((int(line[EventData.userId.value]), 
                            int(line[EventData.sessionId.value]),
                            int(line[EventData.itemInSession.value]),
                            line[EventData.artist.value], 
                            line[EventData.song.value], 
                            line[EventData.firstName.value], 
                            line[EventData.lastName.value]))
            session.execute(bound)
            #session.execute(query)
        except Exception as e:
            print(repr(e))
            print(bound)

In [18]:
# execute query2
try:
    rows = session.execute(query2)
    for row in rows:
        print(row)
except Exception as e:
    print(repr(e))

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [19]:
# define query3
query3 = """
SELECT firstName, lastName 
FROM query3 
WHERE song='All Hands Against His Own'
"""                

In [20]:
# insert data into table to perform query3
# this insert will be slightly different since a user can listen to the same song many times
# the query only cares about finding users who listened to a song
# that means we can use IF NOT EXISTS to only insert an instance of a user listening to a song once
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO query3 (
            song,
            userId,
            firstName,
            lastName
        )
        VALUES (?, ?, ?, ?)
        IF NOT EXISTS
        """
        try:
            s = session.prepare(query)
            bound = s.bind((line[EventData.song.value],
                            int(line[EventData.userId.value]),
                            line[EventData.firstName.value], 
                            line[EventData.lastName.value]))
            session.execute(bound)
        except Exception as e:
            print(repr(e))
            print(bound)

In [21]:
# execute query3
try:
    rows = session.execute(query3)
    for row in rows:
        print(row)
except Exception as e:
    print(repr(e))

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [22]:
# drop tables to clean up keyspace
try:
    session.execute('DROP TABLE query1')
except Exception as e:
    print(e)
try:
    session.execute('DROP TABLE query2')
except Exception as e:
    print(e)
try:
    session.execute('DROP TABLE query3')
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [23]:
# release the connection to the Apache Cassandra instance
session.shutdown()
cluster.shutdown()